In [1]:
import rasterio  
import rasterio.mask
import geopandas as gps
import os
import time
import shutil
from shapely.geometry import Point, Polygon,box
import numpy as np

In [2]:
auxiliary_data_dir=r'/home/nkd/hbh/auxiliary_dataset.gdb'
# grwl_path = os.path.join(auxiliary_data_dir, 'GRWL') 
# OSM_path =  os.path.join(auxiliary_data_dir, 'OSM_water') 
raw_prediction_dir=r'/media/nkd/data18931/result_0130/tif'
output_dir=r'/media/nkd/data18931/result_0130/after_mask'

## 1 land mask

In [26]:
start_id=17001
end_id=start_id+1000
out_dir=os.path.join(output_dir,'{}_{}'.format(start_id,end_id-1))
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
start=time.time()
for index in range(start_id,end_id):
# for index in range(9294,9295):    
    raw_pre_dir=os.path.join(raw_prediction_dir,'{}_{}/pre_{}_iew100.tif'.format(start_id,end_id-1,index))
    out_pre_dir=os.path.join(out_dir,'pre_{}_after_land_mask.tif'.format(index))
    # print(out_pre_dir)
    raw_prediction_image=rasterio.open(raw_pre_dir)
    bbox = box(*raw_prediction_image.bounds)
    meta=raw_prediction_image.meta
    meta.update({"compress": 'lzw'})
    # grwl_river_patch = gps.read_file(auxiliary_data_dir,layer='GRWL_DNRiver' ,bbox=bbox) # data from GRWL_mask_V01.01 product with DN = 255 (River) (see https://doi.org/10.5281/zenodo.1297434).
    # ocean_patch = gps.read_file(auxiliary_data_dir,layer='oceanline_polygon', bbox=bbox) 
    # mask_patch = ocean_patch#grwl_river_patch._append(ocean_patch)
    # mask_patch_geojson = mask_patch.geometry.values 
    land_mask = gps.read_file('/home/nkd/hbh/continent_polygon.shp', bbox=bbox) 
    mask_patch_geojson = land_mask.geometry.values 
 # perform land mask
    if len(mask_patch_geojson) > 0:
        modelpredtictions_masked, out_meta = rasterio.mask.mask(raw_prediction_image, mask_patch_geojson, all_touched=True, invert=False)
        print(f'Mask finished: {index} / {end_id-1}')
        with rasterio.open(out_pre_dir, "w", **meta) as prediction_masked:
            prediction_masked.write(modelpredtictions_masked)
    
end=time.time()
print(f'time:{end-start}')

Mask finished: 17001 / 18000
Mask finished: 17002 / 18000
Mask finished: 17003 / 18000
Mask finished: 17006 / 18000
Mask finished: 17008 / 18000
Mask finished: 17009 / 18000
Mask finished: 17012 / 18000
Mask finished: 17013 / 18000
Mask finished: 17014 / 18000
Mask finished: 17015 / 18000
Mask finished: 17016 / 18000
Mask finished: 17017 / 18000
Mask finished: 17018 / 18000
Mask finished: 17019 / 18000
Mask finished: 17020 / 18000
Mask finished: 17021 / 18000
Mask finished: 17023 / 18000
Mask finished: 17025 / 18000
Mask finished: 17028 / 18000
Mask finished: 17031 / 18000
Mask finished: 17032 / 18000
Mask finished: 17035 / 18000
Mask finished: 17036 / 18000
Mask finished: 17037 / 18000
Mask finished: 17039 / 18000
Mask finished: 17041 / 18000
Mask finished: 17043 / 18000
Mask finished: 17044 / 18000
Mask finished: 17045 / 18000
Mask finished: 17047 / 18000
Mask finished: 17048 / 18000
Mask finished: 17050 / 18000
Mask finished: 17051 / 18000
Mask finished: 17052 / 18000
Mask finished:

In [44]:
for start_id in np.arange(6001,7001,1000):
    end_id=start_id+1000
    out_dir=os.path.join(output_dir,'{}_{}'.format(start_id,end_id-1))
    for root, dirs, files in os.walk(out_dir):
        for file in files:
            old_file=os.path.join(root, file)
            os.rename(old_file,old_file.replace("ocean","land"))

## 2 split into different regions

In [ ]:
region_dir=r'D:\lakemapping\8_postprocess\region'
region_name='africa_2'
old_folder=r'D:\postprocess\tif2shp_dn1'
new_folder=r'D:\postprocess\tif2shp_dn1\{}'.format(region_name)
if not os.path.exists(new_folder):
    os.makedirs(new_folder)
region=gps.read_file(os.path.join(region_dir,region_name+".shp"))
region

In [ ]:
index_list=region.Id.tolist()
for i in index_list:
    for root, dirs, files in os.walk(old_folder):
        for file in files:
            if file.startswith(f'pre_{i}_masked_2shp_dn1'):
                old_file=os.path.join(root, file)
                new_file=os.path.join(new_folder, file)
                shutil.move(old_file,new_file)

In [8]:
ocean=gps.read_file('/home/nkd/hbh/GridShp_ocean.shp')
ocean

,UL_Lon,UL_Lat,Id,Shape_Leng,Shape_Area,geometry
0,-177,2,2,4.0,1.0,"POLYGON ((-177.00000 1.00000, -177.00000 2.000..."
1,-162,6,5,4.0,1.0,"POLYGON ((-162.00000 5.00000, -162.00000 6.000..."
2,-162,7,6,4.0,1.0,"POLYGON ((-162.00000 6.00000, -162.00000 7.000..."
3,-160,5,9,4.0,1.0,"POLYGON ((-160.00000 4.00000, -160.00000 5.000..."
4,-157,2,12,4.0,1.0,"POLYGON ((-157.00000 1.00000, -157.00000 2.000..."
...,...,...,...,...,...,...
892,-36,-55,18916,4.0,1.0,"POLYGON ((-36.00000 -56.00000, -36.00000 -55.0..."
893,69,-50,18922,4.0,1.0,"POLYGON ((69.00000 -51.00000, 69.00000 -50.000..."
894,159,-54,18925,4.0,1.0,"POLYGON ((159.00000 -55.00000, 159.00000 -54.0..."
895,165,-51,18926,4.0,1.0,"POLYGON ((165.00000 -52.00000, 165.00000 -51.0..."


In [10]:
id=ocean.Id.tolist()

In [12]:
for i in id:
    if i<=8000:
        if i<=1000:
            folder='1_1000'
        elif i<=2000:
            folder='1001_2000'
        elif i<=3000:
            folder='2001_3000'
        elif i<=4000:
            folder='3001_4000'
        elif i<=5000:
            folder='4001_5000'
        elif i<=6000:
            folder='5001_6000'
        elif i<=7000:
            folder='6001_7000'
        elif i<=8000:
            folder='7001_8000'
        old_file=r'/media/nkd/data18931/result_0130/after_mask/'+folder+'/pre_{}_after_land_mask.tif'.format(i)
        new_file=old_file.replace(folder,'ocean')
        shutil.move(old_file,new_file)


FileNotFoundError: [Errno 2] No such file or directory: '/media/nkd/data18931/result_0130/after_mask/7001_8000/pre_7087_after_land_mask.tif'